In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [2]:
credit_data=pd.read_csv('credit_data.csv')
print(credit_data.head())

  Status_of_existing_checking_account  Duration_in_month Credit_history  \
0                                 A11                  6            A34   
1                                 A12                 48            A32   
2                                 A14                 12            A34   
3                                 A11                 42            A32   
4                                 A11                 24            A33   

  Purpose  Credit_amount Savings_Account Present_Employment_since  \
0     A43           1169             A65                      A75   
1     A43           5951             A61                      A73   
2     A46           2096             A61                      A74   
3     A42           7882             A61                      A74   
4     A40           4870             A61                      A73   

   Installment_rate_in_percentage_of_disposable_in_come  \
0                                                  4      
1               

In [3]:
credit_data['class']= credit_data['class']-1
credit_data.head()
print(credit_data.head())

  Status_of_existing_checking_account  Duration_in_month Credit_history  \
0                                 A11                  6            A34   
1                                 A12                 48            A32   
2                                 A14                 12            A34   
3                                 A11                 42            A32   
4                                 A11                 24            A33   

  Purpose  Credit_amount Savings_Account Present_Employment_since  \
0     A43           1169             A65                      A75   
1     A43           5951             A61                      A73   
2     A46           2096             A61                      A74   
3     A42           7882             A61                      A74   
4     A40           4870             A61                      A73   

   Installment_rate_in_percentage_of_disposable_in_come  \
0                                                  4      
1               

In [4]:
def IV_calc(data,var):
    if data[var].dtypes == 'object':
        dataf = data.groupby([var])['class'].agg(['count','sum'])
        dataf.columns = ["Total","bad"]
        dataf["good"] = dataf['Total'] - dataf['bad']
        dataf["bad_per"] = dataf["bad"]/dataf["bad"].sum()
        dataf["good_per"] = dataf["good"]/dataf["good"].sum()
        dataf["IV"] = np.log(dataf["good_per"]/dataf["bad_per"]) * (dataf["good_per"] - dataf["bad_per"])
        return dataf
    else:
        data["bin_var"] = pd.qcut(data[var].rank(method='first'),10)
        dataf = data.groupby(['bin_var'])['class'].agg(['count','sum'])
        dataf.columns = ["Total","bad"]
        dataf["good"] = dataf['Total'] - dataf['bad']
        dataf["bad_per"] = dataf["bad"]/dataf["bad"].sum()
        dataf["good_per"] = dataf["good"]/dataf["good"].sum()
        dataf["IV"] = np.log(dataf["good_per"]/dataf["bad_per"]) * (dataf["good_per"] - dataf["bad_per"])
        return dataf

In [6]:
discrete_columns = ['Status_of_existing_checking_account','Credit_history','Purpose','Savings_Account','Present_Employment_since','Personal_status_and_sex','Other_debtors','Property','Other_installment_plans','Housing','Job','Telephone','Foreign_worker']
continuous_columns = ['Duration_in_month','Credit_amount','Installment_rate_in_percentage_of_disposable_in_come','Present_residence_since','Age_in_year','Number_of_existing_credits_at_the_bank','Number_of_people_being_liable_to_provide_maintenance_for']
total_columns = discrete_columns + continuous_columns;


In [7]:
IV_list = []
for col in total_columns:
    assigned_data = IV_calc(data=credit_data,var=col)
    iv_val = round(assigned_data['IV'].sum(),3)
    dt_type = credit_data[col].dtypes
    IV_list.append((iv_val,col,dt_type))

IV_list = sorted(IV_list, reverse = True)
print(len(IV_list))
for i in range(len(IV_list)):
    print(IV_list[i][0], ",", IV_list[i][1],",type=",IV_list[i][2])

20
0.666 , Status_of_existing_checking_account ,type= object
0.293 , Credit_history ,type= object
0.261 , Duration_in_month ,type= int64
0.196 , Savings_Account ,type= object
0.169 , Purpose ,type= object
0.113 , Property ,type= object
0.113 , Credit_amount ,type= int64
0.103 , Age_in_year ,type= int64
0.098 , Number_of_existing_credits_at_the_bank ,type= int64
0.086 , Present_Employment_since ,type= object
0.083 , Housing ,type= object
0.062 , Installment_rate_in_percentage_of_disposable_in_come ,type= int64
0.058 , Other_installment_plans ,type= object
0.049 , Present_residence_since ,type= int64
0.045 , Personal_status_and_sex ,type= object
0.044 , Foreign_worker ,type= object
0.034 , Number_of_people_being_liable_to_provide_maintenance_for ,type= int64
0.032 , Other_debtors ,type= object
0.009 , Job ,type= object
0.006 , Telephone ,type= object


In [8]:
dummy_stseca =pd.get_dummies(credit_data['Status_of_existing_checking_account'],prefix='status_exs_accnt')
dummy_ch = pd.get_dummies(credit_data['Credit_history'],prefix='cred_hist')
dummy_purpose = pd.get_dummies(credit_data['Purpose'],prefix='purpose')
dummy_savacc = pd.get_dummies(credit_data['Savings_Account'],prefix='sav_acc')
dummy_presc = pd.get_dummies(credit_data['Present_Employment_since'],prefix='pre_emp_snc')
dummy_perssx = pd.get_dummies(credit_data['Personal_status_and_sex'],prefix='per_stat_sx')
dummy_property = pd.get_dummies(credit_data['Property'],prefix='property')
dummy_othinstpln =pd.get_dummies(credit_data['Other_installment_plans'],prefix='oth_inst_pln')
dummy_forgnwrkr = pd.get_dummies(credit_data['Foreign_worker'],prefix='forgn_wrkr')
dummy_othdts = pd.get_dummies(credit_data['Other_debtors'],prefix='oth_debtors')

In [14]:
continuous_columns = ['Duration_in_month', 'Credit_amount','Installment_rate_in_percentage_of_disposable_in_come', 'Age_in_year','Number_of_existing_credits_at_the_bank']
credit_continuous = credit_data[continuous_columns]
credit_data_new = pd.concat([dummy_stseca,dummy_ch,dummy_purpose,dummy_savacc, dummy_presc,dummy_perssx, dummy_property,dummy_othinstpln,dummy_othdts,dummy_forgnwrkr,credit_continuous,credit_data['class']],axis=1)

In [15]:
x_train,x_test,y_train,y_test = train_test_split(credit_data_new.drop(['class'],axis=1),credit_data_new['class'],train_size=0.7,random_state=42)
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [17]:
remove_cols_extra_dummy = ['status_exs_accnt_A11', 'cred_hist_A30',
'purpose_A40', 'sav_acc_A61','pre_emp_snc_A71','per_stat_sx_A91',
'oth_debtors_A101','property_A121', 'oth_inst_pln_A141','forgn_wrkr_A201']

remove_cols_insig = []
remove_cols = list(set(remove_cols_extra_dummy + remove_cols_insig))

In [18]:
import statsmodels.api as sm
logistic_model = sm.Logit(y_train,sm.add_constant(x_train.drop(remove_cols,axis =1))).fit()
print(logistic_model.summary())

Optimization terminated successfully.
         Current function value: 0.456186
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                  class   No. Observations:                  700
Model:                          Logit   Df Residuals:                      659
Method:                           MLE   Df Model:                           40
Date:                Mon, 20 Apr 2020   Pseudo R-squ.:                  0.2517
Time:                        04:43:55   Log-Likelihood:                -319.33
converged:                       True   LL-Null:                       -426.75
                                        LLR p-value:                 8.624e-26
                                                           coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------------------------------------------------
const                   

In [19]:
print("\n Variance Inflation Factor")
cnames = x_train.drop(remove_cols,axis=1).columns 
for i in np.arange(0, len(cnames)):
    xvars = list(cnames)
    yvar = xvars.pop(i)
    mod = sm.OLS(x_train.drop(remove_cols,axis=1)[yvar], sm.add_constant(x_train.drop(remove_cols,axis=1)[xvars]))
    res = mod.fit()
    vif = 1/(1-res.rsquared)
    print(yvar, round(vif,3))


 Variance Inflation Factor
status_exs_accnt_A12 1.722
status_exs_accnt_A13 1.24
status_exs_accnt_A14 1.823
cred_hist_A31 2.214
cred_hist_A32 6.991
cred_hist_A33 2.894
cred_hist_A34 5.997
purpose_A41 1.421
purpose_A410 1.213
purpose_A42 1.545
purpose_A43 1.754
purpose_A44 1.078
purpose_A45 1.116
purpose_A46 1.201
purpose_A48 1.097
purpose_A49 1.484
sav_acc_A62 1.141
sav_acc_A63 1.103
sav_acc_A64 1.109
sav_acc_A65 1.179
pre_emp_snc_A72 3.3
pre_emp_snc_A73 4.403
pre_emp_snc_A74 3.368
pre_emp_snc_A75 3.91
per_stat_sx_A92 6.309
per_stat_sx_A93 6.948
per_stat_sx_A94 3.227
property_A122 1.56
property_A123 1.741
property_A124 1.784
oth_inst_pln_A142 1.321
oth_inst_pln_A143 1.395
oth_debtors_A102 1.118
oth_debtors_A103 1.195
forgn_wrkr_A202 1.127
Duration_in_month 2.006
Credit_amount 2.455
Installment_rate_in_percentage_of_disposable_in_come 1.336
Age_in_year 1.378
Number_of_existing_credits_at_the_bank 1.559


In [20]:
y_pred = pd.DataFrame(logistic_model.predict(sm.add_constant(x_train.drop(remove_cols,axis=1))))
y_pred.columns = ["probs"]
both = pd.concat([y_train,y_pred],axis=1)

In [27]:
zeros = both[['class','probs']][both['class']==0]
ones = both[['class','probs']][both['class']==1]
def df_crossjoin(df1, df2, **kwargs):
    df1['_tmpkey'] = 1
    df2['_tmpkey'] = 1
    res = pd.merge(df1, df2, on='_tmpkey', **kwargs).drop('_tmpkey',axis=1)
    res.index = pd.MultiIndex.from_product((df1.index, df2.index))
    df1.drop('_tmpkey', axis=1, inplace=True)
    df2.drop('_tmpkey', axis=1, inplace=True)
    return res


In [29]:
joined_data = df_crossjoin(ones,zeros)
print(joined_data)

         class_x   probs_x  class_y   probs_y
1   0        1.0  0.206071      0.0  0.199146
    6        1.0  0.206071      0.0  0.847087
    8        1.0  0.206071      0.0  0.682558
    12       1.0  0.206071      0.0  0.191265
    14       1.0  0.206071      0.0  0.043020
    16       1.0  0.206071      0.0  0.108531
    17       1.0  0.206071      0.0  0.014099
    19       1.0  0.206071      0.0  0.675033
    20       1.0  0.206071      0.0  0.552484
    21       1.0  0.206071      0.0  0.262825
    22       1.0  0.206071      0.0  0.146062
    24       1.0  0.206071      0.0  0.330074
    26       1.0  0.206071      0.0  0.169577
    27       1.0  0.206071      0.0  0.302267
    28       1.0  0.206071      0.0  0.871294
    32       1.0  0.206071      0.0  0.734678
    34       1.0  0.206071      0.0  0.030118
    36       1.0  0.206071      0.0  0.733428
    38       1.0  0.206071      0.0  0.244099
    40       1.0  0.206071      0.0  0.393004
    41       1.0  0.206071      0.

In [30]:
joined_data['concordant_pair']=0
joined_data.loc[joined_data['probs_x'] > joined_data['probs_y'], 'concordant_pair'] = 1
joined_data['discordant_pair']=0
joined_data.loc[joined_data['probs_x'] < joined_data['probs_y'], 'discordant_pair'] = 1
joined_data['tied_pair'] = 0
joined_data.loc[joined_data['probs_x'] == joined_data['probs_y'], 'tied_pair'] = 1
print(joined_data.shape[0])

102619


In [31]:
p_conc = (joined_data['concordant_pair'].sum())/joined_data.shape[0]
p_disc = (joined_data['discordant_pair'].sum())/(joined_data.shape[0])
c_statistics = 0.5 + (p_conc - p_disc)/2.0
print("\nC-statistics:", round(c_statistics,4))


C-statistics: 0.5125


In [32]:
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import auc
fpr, tpr, thresholds = metrics.roc_curve(both['class'],both['probs'], pos_label=1)
roc_auc = auc(fpr,tpr)
plt.figure 
lw = 2
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area =%0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (1-Specificity)')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve - German Credit Data')
plt.legend(loc="lower right")
plt.show()

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [34]:
both['probs']

0      0.199146
1      0.206071
2      0.288691
3      0.099776
4      0.021924
5      0.090936
6      0.847087
7      0.225858
8      0.682558
9      0.008902
10     0.386405
11     0.655979
12     0.191265
13     0.751658
14     0.043020
15     0.074990
16     0.108531
17     0.014099
18     0.820619
19     0.675033
20     0.552484
21     0.262825
22     0.146062
23     0.053715
24     0.330074
25     0.012188
26     0.169577
27     0.302267
28     0.871294
29     0.297900
         ...   
956         NaN
957         NaN
958         NaN
960         NaN
961         NaN
962         NaN
964         NaN
965         NaN
966         NaN
967         NaN
968         NaN
969         NaN
970         NaN
971         NaN
972         NaN
976         NaN
979         NaN
980         NaN
981         NaN
982         NaN
983         NaN
984         NaN
987         NaN
988         NaN
990         NaN
992         NaN
993         NaN
996         NaN
997         NaN
999         NaN
Name: probs, dtype: floa

In [4]:
a = np.random.randn(3, 3)
b = np.random.randn(3, 1)
c = a*b
print(c)

[[-2.69061986  0.89977408  1.64481672]
 [ 0.58473438  1.59408041 -1.71484112]
 [-0.58885867 -1.06390362  2.29890119]]


In [5]:
A = np.random.randn(4,3)
B = np.sum(A, axis = 1, keepdims = True)

In [7]:
print(B.shape)

(4, 1)


In [8]:
a= np.random.randn(5, 3)
a

array([[ 0.58288027,  0.33497139,  1.47040882],
       [-0.65056327,  0.0166006 , -0.2544    ],
       [-0.67535331, -1.30938326, -1.85857936],
       [-0.39843007,  0.00963093,  0.95712233],
       [-1.68843272, -1.85698817, -0.6732123 ]])

In [9]:
L=10
for l in reversed(range(L-1)):
    print(l)

8
7
6
5
4
3
2
1
0


In [10]:
L=10
for l in (range(L)):
    print(l)

0
1
2
3
4
5
6
7
8
9
